In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
for cm in ChapterMark.objects.all():
    print(cm.audio)

1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
2 - asdf
2 - asdf
2 - asdf


<IPython.core.display.Javascript object>